In [2]:
import os
import time
from typing import Dict, List

import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from pandas import DataFrame
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from utils.const import *

load_dotenv()

True

In [3]:
def init_cli():
    mjs = """
    _________.__              .__  .__  __  .__        ___________.__
    ____________________  _____       _____  __________________________   
    \______   \______   \/  _  \     /     \ \_   _____|__    ___/  _  \  
     |       _/|     ___/  /_\  \   /  \ /  \ |    __)_  |    | /  /_\  \ 
     |    |   \|    |  /    |    \ /    Y    \|        \ |    |/    |    \\
     |____|_  /|____|  \____|__  / \____|__  /_______  / |____|\____|__  /
            \/                 \/          \/        \/                \/ 
   """
    print(mjs)

In [4]:
# import time
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from bs4 import BeautifulSoup

# def visit_multiple_pages(initial_page: int, final_page: int):
#     # Inicializar el driver
#     driver = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)
#     # Abrir Chrome
#     URL = 'https://google.com'
#     driver.get(URL)
#     # Buscar el input
#     input_search = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "q")))
#     # Escribir en el input
#     input_search.send_keys('python')
#     # Hacer clic en el botón de búsqueda
#     input_search.submit()

#     # Cambiar de página con el botón "Siguiente"
#     for _ in range(initial_page, final_page):
#         # Esperar a que cargue la página
#         time.sleep(5)
#         # Obtener el HTML de la página actual
#         html = driver.page_source
#         # Procesar el HTML
#         soup = BeautifulSoup(html, 'html.parser')
#         # Obtener los enlaces
#         links = soup.find_all('a', href=True)
#         # Obtener los enlaces de la página actual
#         links = [link['href'] for link in links if link['href'].startswith('https://')]
#         # Hacer clic en el botón "Siguiente"
#         next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'pnnext')))
#         next_button.click()

#     # Cerrar el navegador al finalizar
#     driver.quit()

# visit_multiple_pages(1, 3)

In [16]:
def getExperienceData(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    sections = soup.find_all('div', id='experience')
    if not sections:
        return None
    #secctions next sibling
    next_sibling = sections[0].find_next_sibling()
    #second next sibling
    second_next_sibling = next_sibling.find_next_sibling()
    experiencias = []
    sections = second_next_sibling.find_all('li', class_='artdeco-list__item')
    for section in sections:
        duracion = section.find('span', class_='t-14 t-normal t-black--light').get_text(strip=True)
        empresa = section.find('span', class_='t-14 t-normal').get_text(strip=True)
        cargo = section.find('div', class_='display-flex align-items-center mr1 t-bold').get_text(strip=True)
        actividades = section.find('div', class_='FKFqOBZeKsYhiuZKTYcfkNJIpRwfptzrAEOM').get_text(strip=True)
        experiencias.append({
            'duración': duracion,
            'empresa': empresa,
            'cargo': cargo,
            'actividades': actividades
        })
    return experiencias

In [5]:
def get_one_profile(driver,curren_url: str):
    # obtener el codigo de la pagina
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')
    print(soup.prettify())
    #Obtener el ur de la imagen
    div = soup.find('div', class_='pv-top-card__non-self-photo-wrapper ml0')
    img = div.find('img')['src'] if div else "No hay imagen"

    

    nombre = soup.find(xpath='//*[@id="ember53"]/div[2]/div[2]/div/div[1]/div[1]/div[1]/div[1]/div/h1')
    print(f"el nombre es: {nombre}")
    nombre = nombre.text.strip() if nombre else "No hay nombre"
    description = soup.find('div', class_= 'text-body-medium break-words')
    description = description.text.strip() if description else "No hay descripcion"
    empresa_actual = soup.find('div', class_='inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp inline')
    empresa_actual = empresa_actual.text.strip() if empresa_actual else "No hay empresa actual"
    about = soup.find('section', class_='artdeco-card ember-view relative break-words pb3 mt2')
    if about:
      about = about.find('div', class_='inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp full-width')
      about = about.text.strip() if about else "No hay about"
      about = about.replace("… ver más", "")
    
    
    conocimientos_aptitudes = soup.find_all('div', class_='pvs-entity pvs-entity--padded pvs-list__item--no-padding-in-columns')
    conocimiento = [{
      "nombre": conocimiento_aptitud.find('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold').text.strip() if conocimiento_aptitud.find('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold') else "No hay nombre",
      "descripcion": conocimiento_aptitud.find('div', class_='inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp full-width').text.strip() if conocimiento_aptitud.find('div', class_='inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp full-width') else "No hay descripcion",
    }
    for conocimiento_aptitud in conocimientos_aptitudes
    ]
    info_idiomas = soup.find_all('section', class_='artdeco-card ember-view relative break-words pb3 mt2')
    info_idiomas = [idioma for idioma in info_idiomas if idioma.find('div', id='languages')]
    idiomas = []
    for idioma in info_idiomas:
        items = idioma.find_all('li', class_='artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column')
        for item in items:
            nombre_idioma = item.find('div', class_='display-flex align-items-center mr1 t-bold').text.strip() if item.find('div', class_='display-flex align-items-center mr1 t-bold') else "No hay idioma"
            nivel = item.find('span', class_='t-14 t-normal t-black--light').text.strip() if item.find('span', class_='t-14 t-normal t-black--light') else "No hay nivel"
            idioma_data = {
                "idioma": nombre_idioma,
                "nivel": nivel
            }
            idiomas.append(idioma_data)
    educaciones = soup.find_all('section', class_='artdeco-card ember-view relative break-words pb3 mt2')
    educaciones = [edu for edu in educaciones if edu.find('div', id='education')]
    educacion = []
    for edu in educaciones:
        items = edu.find_all('li', class_='artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column')
        for item in items:
            img = item.find('img')['src'] if item.find('img') else "No hay imagen"
            escuela = item.find('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold').text.strip() if item.find('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold') else "No hay nombre"
            especialidad = item.find('span', class_='t-14 t-normal').text.strip() if item.find('span', class_='pv-entity__secondary-title pv-entity__fos t-14 t-black t-normal') else "No hay especialidad"
            educacion_data = {
                "img": img,
                "escuela": escuela,
                "especialidad": especialidad
            }
            
            educacion.append(educacion_data)
  
    licencias_certificaciones = soup.find_all('section', class_='artdeco-card ember-view relative break-words pb3 mt2')
    licencias = []
    for licencia in licencias_certificaciones:
        if licencia.find('div', id='licenses_and_certifications'):
            items = licencia.find_all('li', class_='artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column')
            for item in items:
                img = item.find('img')['src'] if item.find('img') else "No hay imagen"
                nombre = item.find('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold').text.strip() if item.find('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold') else "No hay nombre"
                plataforma = item.find('span', class_='t-14 t-normal').text.strip() if item.find('span', class_='t-14 t-normal') else "No hay plataforma"
                expedicion = item.find('span', class_='t-14 t-normal t-black--light').text.strip() if item.find('span', class_='t-14 t-normal t-black--light') else "No hay expedicion"
                URL_certificacion = item.find('a')['href'] if item.find('a') else "No hay URL"
                licencia_data = {
                    "img": img,
                    "nombre": nombre,
                    "plataforma": plataforma,
                    "expedicion": expedicion,
                    "URL_certificacion": URL_certificacion
                }
                licencias.append(licencia_data)
    info = {
       "url": curren_url,
        "nombre": nombre,
        "img": img,
        "description": description,
        "empresa_actual": empresa_actual,
        "about": about if about else "No hay about",
        "idiomas": idiomas if idiomas else "No hay idiomas",
        "conocimientos_aptitudes" : conocimiento if conocimiento else "No hay conocimientos y aptitudes",
        "educacion": educacion if educacion else "No hay educacion",
        "licencias_certificaciones": licencias if licencias else "No hay licencias y certificaciones"
    }
    return info

In [6]:
def scrap_data_from_profile(driver,urls_profiles: List[str]):
  print("Extrayendo datos de los perfiles")
  data_profiles = []
  for url in urls_profiles:
      driver.get(url)
      #imprimir el html de la pagina

      print(f"Extrayendo datos de {url}")
      print(driver.page_source)
      #mandar el html a un archivo
      with open('profile.html', 'w') as file:
        file.write(driver.page_source)
        

      try:
         WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'application-outlet')))
      except Exception as e:
        print(e)
        print(f"Error en {url}")
        time.sleep(5)
        continue
      #ir hasta abajo de la pagina
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      time.sleep(5)
      #obtener la informacion del perfil
      print("Obteniendo la informacion del perfil")
      data_profiles.append(get_one_profile(driver,url))
  return data_profiles

In [7]:
def search_by_keyword(driver, config: Dict[str, str], urls_profiles: List[str]):
    keywords = config["keywords"]
    location = config["location"]
    initial_page = config["initial_page"]
    final_page = config["final_page"]
    
    # Abrir la página de búsqueda
    search_url = "https://www.google.com"
    driver.get(search_url)

    # Esperar hasta que el campo de entrada de búsqueda esté presente en la página
    input_search = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "q")))

    input_search.send_keys(f"site:linkedin.com/in/ AND {keywords} AND {location}")
    input_search.submit()

    for _ in range(initial_page, final_page + 1):
        # Esperar hasta que aparezca el elemento de resultados de búsqueda en la página
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'rcnt')))

        # Obtener el código fuente de la página de resultados de búsqueda
        src = driver.page_source

        # Pasear el HTML
        soup = BeautifulSoup(src, 'lxml')
        profiles = soup.find_all('div', class_='MjjYud')

        print(f"Se encontraron {len(profiles)} perfiles")
        
        for profile in profiles:
            link_tag = profile.find('a')
            if link_tag is not None:  # Verificar si link_tag no es None
                profile_linkedin_url = link_tag['href']
                try:
                    if profile_linkedin_url.startswith('/search'):
                        break
                    urls_profiles.append(profile_linkedin_url)
                except Exception as e:
                    print(e)
                    print(f"Error en {profile_linkedin_url}")
            else:
                print("No se encontró una etiqueta <a> en este perfil:", profile)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Saber si hay botón de siguiente o scroll infinito
        try:
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'pnnext')))
            next_button.click()
        except Exception as e:
            print(e)
            break

    info = scrap_data_from_profile(driver, urls_profiles)
    df = pd.DataFrame(info)
    return df

In [8]:
def search_by_urls(driver,urls_profiles: List[str]):
    info = scrap_data_from_profile(driver, urls_profiles)
    print(info)
    df = pd.DataFrame(info)

    return df

In [12]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from storage.mysql.adapter import MySQLAdapter
from storage.sqlite.adapter import SQLiteAdapter
from model.LinkedinProfile import LinkedInProfile
import pandas as pd
from pandas import DataFrame


def save_profiles(profiles: DataFrame, adapter):
    try:
        adapter.connect()
        profiles = profiles.where(pd.notnull(profiles), None)
        profiles = profiles.to_dict(orient='records')
        profiles = [LinkedInProfile(**profile) for profile in profiles]
        adapter.saveMany(profiles)
        adapter.close() 
        
        return True
    except Exception as e:
        print(e)
        return False

class scraper:  
    def search_people(config) -> DataFrame:
      init_cli()
      
      is_search:bool = config["is_search"]
      profiles_to_search:List[str] = config["profiles_to_search"]
      urls_profiles = []
      df:DataFrame = pd.DataFrame()

      options = Options()
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      options.add_argument('--disable-dev-shm-usage')
      driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

      print(f"launching {LOGIN_URL}")
      # Logging into LinkedIn
      driver.get("https://www.linkedin.com/login")
      time.sleep(2)

      user = "meta.dev@forteinnovation.mx"
      password = "m3t4.dev-321"

      print(f"Logging in with {user} and {password}")

      username = driver.find_element(By.ID, "username")
      username.send_keys(user)

      pword = driver.find_element(By.ID, "password")
      pword.send_keys(password)

      driver.find_element(By.XPATH, "//button[@type='submit']").click()


      if is_search:
        print("Buscando por urls")
        df = search_by_urls(driver,profiles_to_search)
      else:
        df = search_by_keyword(driver,config,urls_profiles)
      driver.quit()
      return df
    

if __name__ == "__main__":
    config = {
        "is_search": False,
        "profiles_to_search": [
            "https://www.linkedin.com/in/susana-rojas-761102282/",
            "https://www.linkedin.com/in/yuridia-molina-52090a305/",
            "https://www.linkedin.com/in/orlando-alexis-flores-dom%C3%ADnguez-571047201/",
            "https://www.linkedin.com/in/carlos-rodriguez-b1a5b4290/"
        ],
        "keywords": "python",
        "location": "Mexico",
        "initial_page": 1,
        "final_page": 3
    }
    df = scraper.search_people(config)
    print(df)
    df.to_csv("data.csv", index=False)
    adapter = MySQLAdapter()
    save_profiles(df, adapter)
    









NameError: name 'init_cli' is not defined

In [ ]:
# #run scraper
# from model.config import ConfigScrap



# # basic_config = {
# #     "keywords": "diseñador",
# #     "location": "celaya",
# # }

# # medium_config = {
# #     "keywords": "react",
# #     "location": "celaya",
# #     "initial_page": 1,
# #     "final_page": 2
# # }

# search_config = {
#     "profiles_to_search": ["https://mx.linkedin.com/in/nanadiseno","https://mx.linkedin.com/in/luisfernando-celaya"],
#     "is_search": True,
# }

# conf = ConfigScrap(**search_config)



# df = scraper.search_people(conf.__dict__)


# if df is not None:
#   # Create a CSV file and save the information
#   df.to_csv("data.csv", index=False)
# else:
#   print("No hay datos")


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Configuración de opciones para el navegador
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-extensions')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Iniciar sesión en LinkedIn
driver.get("https://www.linkedin.com/login")
time.sleep(2)

user = "meta.dev@forteinnovation.mx"
password = "m3t4.dev-321"

username = driver.find_element(By.ID, "username")
username.send_keys(user)

pword = driver.find_element(By.ID, "password")
pword.send_keys(password)

driver.find_element(By.XPATH, "//button[@type='submit']").click()

# Navegar al perfil de Maura Ledezma
driver.get("https://www.linkedin.com/in/daniel-ruiz-guti%C3%A9rrez-3a925b65/")
time.sleep(2)

#esperar 5 segundos para que cargue la pagina
time.sleep(5)

# Encontrar el nombre de la persona usando XPath
nombre = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[1]/span[1]/a/h1')
print(nombre.text)

source = BeautifulSoup(driver.page_source, 'lxml')


def getExperienceData(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    sections = soup.find_all('div', id='experience')
    if not sections:
        return None
    #secctions next sibling
    next_sibling = sections[0].find_next_sibling()
    #second next sibling
    second_next_sibling = next_sibling.find_next_sibling()
    experiencias = []
    sections = second_next_sibling.find_all('li', class_='artdeco-list__item')
    for section in sections:
        duracion = section.find('span', class_='t-14 t-normal t-black--light').get_text(strip=True)
        empresa = section.find('span', class_='t-14 t-normal').get_text(strip=True)
        cargo = section.find('div', class_='display-flex align-items-center mr1 t-bold').get_text(strip=True)
        actividades = section.find('div', class_='FKFqOBZeKsYhiuZKTYcfkNJIpRwfptzrAEOM').get_text(strip=True)
        experiencias.append({
            'duración': duracion,
            'empresa': empresa,
            'cargo': cargo,
            'actividades': actividades
        })
    return experiencias


def getEducationData(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    sections = soup.find_all('div', id='education')
    if not sections:
        return None
    #secctions next sibling
    next_sibling = sections[0].find_next_sibling()
    #second next sibling
    second_next_sibling = next_sibling.find_next_sibling()
    educaciones = []
    sections = second_next_sibling.find_all('li', class_='artdeco-list__item')
    for section in sections:
        duracion = section.find('span', class_='pvs-entity__caption-wrapper').get_text(strip=True) if section.find('span', class_='pvs-entity__caption-wrapper') else "No hay duración"
        carrera = section.find('span', class_='t-14 t-normal').get_text(strip=True) if section.find('span', class_='t-14 t-normal') else "No hay escuela"
        escuela = section.find('div', class_='display-flex align-items-center mr1 t-bold').text.strip() if section.find('div', class_='display-flex align-items-center mr1 t-bold') else "No hay carrera"
        educaciones.append({
            'duración': duracion,
            'escuela': escuela,
            'carrera': carrera
        })
    return educaciones


def getLicencias_certificacionesData(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    sections = soup.find_all('div', id='licenses_and_certifications')
    if not sections:
        return None
    #secctions next sibling
    next_sibling = sections[0].find_next_sibling()
    #second next sibling
    second_next_sibling = next_sibling.find_next_sibling()
    licencias_certificaciones = []
    sections = second_next_sibling.find_all('li', class_='artdeco-list__item')
    for section in sections:
        img = section.find('img')['src'] if section.find('img') else "No hay imagen"
        nombre = section.find('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold').get_text(strip=True) if section.find('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold') else "No hay nombre"
        plataforma = section.find('span', class_='t-14 t-normal').get_text(strip=True) if section.find('span', class_='t-14 t-normal') else "No hay plataforma"
        expedicion = section.find('span', class_='t-14 t-normal t-black--light').get_text(strip=True) if section.find('span', class_='t-14 t-normal t-black--light') else "No hay expedicion"
        URL_certificacion = section.find('a')['href'] if section.find('a') else "No hay URL"
        licencias_certificaciones.append({
            'img': img,
            'nombre': nombre,
            'plataforma': plataforma,
            'expedicion': expedicion,
            'URL_certificacion': URL_certificacion
        })
    return licencias_certificaciones

certificaciones = getLicencias_certificacionesData(driver)

cookies = driver.get_cookies()

print(cookies)

print(certificaciones)



Daniel Ruiz Gutiérrez
[{'domain': '.linkedin.com', 'expiry': 1721847340, 'httpOnly': False, 'name': 'UserMatchHistory', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': 'AQKpGi3aPkXLEQAAAZBLmpVeVnhHj4LB00a19IXMDwVjJfZbMmn1pPdlgPNmXNiCLY7XDSErMk_tK1P1AJYKgeZtE0PrwksRxjkcmygBUd7SFNok4jl5bL1-wiMVHR9PH8t6JhvcRfFHTRisl-gsUX9kZqAKsLHACVSSSI1F4BrAGQE8MaLcBOJa_haxKje_r7AdrkaNLV-S6VV6uagn6Kzr_IrARekeEm7tkaKvQH3jbgqqVDnz9ao2d_nJ5iZs0Ug3gKFrik2VDBdBeq4YBtIP7Ub_TEHzrnZFWwo4JsblowhoUPlkJT-WJtqsoi1NhNE2FFoW6uZlE57MXgKUvkIHqkDHfjkm6HJpGkFwto617e9__w'}, {'domain': '.linkedin.com', 'expiry': 1721847337, 'httpOnly': False, 'name': 'lms_analytics', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': 'AQF4H60Ax4gQ1QAAAZBLmolAMyWd8QCAd6kMx5dTUfLFuDrTR95od1mGc27y50Ii3jvaHwmxmXM34irhDczMIl9nanY8hfup'}, {'domain': '.linkedin.com', 'expiry': 1750791337, 'httpOnly': True, 'name': 'dfpfpt', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '71589102f2bd451dbb135e9f860566d2'}, {'domain': '.l

In [ ]:
# from pyOutlook import OutlookAccount

# token = "eyJ0eXAiOiJKV1QiLCJub25jZSI6IjFHVWVoY0hjTF9OQXRhWnpkaGhMazhlN1dGUWw3OUpsSGc0bWFON0RRT00iLCJhbGciOiJSUzI1NiIsIng1dCI6IkwxS2ZLRklfam5YYndXYzIyeFp4dzFzVUhIMCIsImtpZCI6IkwxS2ZLRklfam5YYndXYzIyeFp4dzFzVUhIMCJ9.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTAwMDAtYzAwMC0wMDAwMDAwMDAwMDAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC84MzBjZjUwNi0wZjUwLTQxYjQtYTM4Ni01NzRhNDViZDYyY2YvIiwiaWF0IjoxNzE2Mzk0NTI5LCJuYmYiOjE3MTYzOTQ1MjksImV4cCI6MTcxNjM5OTI1MiwiYWNjdCI6MCwiYWNyIjoiMSIsImFpbyI6IkFUUUF5LzhXQUFBQXpNaHk1NExmcFBEZWNhSUdJK1NBckd2Q3BFQldxN2RkNTVRS1pkTnZjZWZSZGwyc2d6a0RyQWlvT1VYdGUzK1giLCJhbXIiOlsicHdkIl0sImFwcF9kaXNwbGF5bmFtZSI6IlBydWViYXMgUHl0aG9uIiwiYXBwaWQiOiJkMDRmY2RlZC05ZWQyLTQ5NWMtOWZiYS01ZGVmYjliOTczOWYiLCJhcHBpZGFjciI6IjEiLCJmYW1pbHlfbmFtZSI6Ik1ldGFwaG9yY2UiLCJnaXZlbl9uYW1lIjoiRGVzYXJyb2xsbyIsImlkdHlwIjoidXNlciIsImlwYWRkciI6IjE4Ny4xODkuMjQ4LjE4OSIsIm5hbWUiOiJEZXNhcnJvbGxvIE1ldGFwaG9yY2UiLCJvaWQiOiI0MzkxOTlmNS1jYWJkLTQ0NzctYmUzNC0zNDg0Yjk4MWUyOGIiLCJwbGF0ZiI6IjMiLCJwdWlkIjoiMTAwMzIwMDM3NjI2NkMyNyIsInJoIjoiMC5BU2tBQnZVTWcxQVB0RUdqaGxkS1JiMWl6d01BQUFBQUFBQUF3QUFBQUFBQUFBQXBBS2MuIiwic2NwIjoiVXNlci5SZWFkIFVzZXIuUmVhZEJhc2ljLkFsbCBwcm9maWxlIG9wZW5pZCBlbWFpbCIsInNpZ25pbl9zdGF0ZSI6WyJrbXNpIl0sInN1YiI6InpnRC03TnhMWFhNWUNJS1J3MnlPaWNHNDBjUWQ0dFU3T0xqeU9aNlVLTkUiLCJ0ZW5hbnRfcmVnaW9uX3Njb3BlIjoiTkEiLCJ0aWQiOiI4MzBjZjUwNi0wZjUwLTQxYjQtYTM4Ni01NzRhNDViZDYyY2YiLCJ1bmlxdWVfbmFtZSI6Im1ldGEuZGV2QGZvcnRlaW5ub3ZhdGlvbi5teCIsInVwbiI6Im1ldGEuZGV2QGZvcnRlaW5ub3ZhdGlvbi5teCIsInV0aSI6Im52WUFYa1QtWTBLbFI0b1pNM2t1QUEiLCJ2ZXIiOiIxLjAiLCJ3aWRzIjpbImI3OWZiZjRkLTNlZjktNDY4OS04MTQzLTc2YjE5NGU4NTUwOSJdLCJ4bXNfc3QiOnsic3ViIjoiQnFHeEZURXptdnd6am1ITmVCM1hmWlF6YVhrNmpTd09neUJuaUtMT00wayJ9LCJ4bXNfdGNkdCI6MTUxMjU4ODYyOH0.l_apZZdwNzkIHq39Y-dfE6wvXCkWhs300IeGUbqSk0r7GGuXJVYTFZa5Xw7XFpB55wAtVpNtvv4WX1e9GB-Of1hoxqEUdDW0Mro9HTVi5xlw9Qr9oQyNVY_rDO0xXecMKMW-lF1O4fG2tA5Z7ZJd1fzkDKhM3r6B8eWiYQh2FCynTGZrq7AVnfwUEChP5yS4cbdU0YJSb9ktOUHAANNen1NkbrVB2WWXDLjGU5ikt-mZCGcFp1Aha4GNpV90lVCectAk00WeiDioY85FjlQdY6e1dqCv3_cBpPtIZ3juqNfBBZjbufBmKFUQ6LoXLku_1sz7EZKmpNDjfjjVK8x9Dw"
# outlook = OutlookAccount(access_token=token)

# # Get the first 10 messages in the inbox
# messages = outlook.get_messages(10)
# print(messages)


ModuleNotFoundError: No module named 'pyOutlook'

In [ ]:
# from storage.mysql.adapter import MySQLAdapter
# from storage.sqlite.adapter import SQLiteAdapter
# from model.LinkedinProfile import LinkedInProfile
# import pandas as pd
# from pandas import DataFrame

# def save_profiles_to_db_safe(profiles: DataFrame):
#     try:
#         adapter = MySQLAdapter()
#         adapter.connect()
#         profiles = profiles.where(pd.notnull(profiles), None)
#         profiles = profiles.to_dict(orient='records')
#         profiles = [LinkedInProfile(**profile) for profile in profiles]
#         adapter.saveMany(profiles)
#         adapter.close()
#         return True
#     except Exception as e:
#         print(e)
#         return False
    

# def save_profiles(profiles: DataFrame, adapter):
#     try:
#         adapter.connect()
#         profiles = profiles.where(pd.notnull(profiles), None)
#         profiles = profiles.to_dict(orient='records')
#         profiles = [LinkedInProfile(**profile) for profile in profiles]
#         adapter.saveMany(profiles)
#         adapter.close()
#         return True
#     except Exception as e:
#         print(e)
#         return False

# adapter = MySQLAdapter()
# adapter.connect()

# data = {
#         'url': [
#             'https://mx.linkedin.com/in/guillermo-izquierdo-colin-09586a49',
#             'https://mx.linkedin.com/in/desarollador-python-525b37274',
#             'https://mx.linkedin.com/in/fridaruh',
#             'https://mx.linkedin.com/in/juan-gerardo-lopez-garcia-58b056150'
#         ],
#         'nombre': ['Guillermo Izquierdo Colin', 'Desarollador Python', 'Frida Ruh', 'Juan Gerardo López García'],
#         'img': [
#             'https://media.licdn.com/dms/image/C5603AQE9pumAAxK1mA/profile-displayphoto-shrink_200_200/0/1607035280376?e=1723680000&v=beta&t=EHNkYIHw37sVXACqo9R9-96ckBIxCCExXf2cnzPlR00',
#             'https://media.licdn.com/dms/image/D4E03AQGlOCb0zBpxAg/profile-displayphoto-shrink_200_200/0/1706061172573?e=1723680000&v=beta&t=kGcmIrfDIaJFkr-oWumOPeJntQFJEq4ETlRBOQZOwC8',
#             'https://media.licdn.com/dms/image/D4E03AQH1KM8slrjxHA/profile-displayphoto-shrink_200_200/0/1708842967664?e=1723680000&v=beta&t=y7dbWxL3bdAgd_QD-1ZztLodEGBPqIA_eJ4yf4aj3sQ',
#             'https://media.licdn.com/dms/image/D4D03AQENVUF03E9RRQ/profile-displayphoto-shrink_200_200/0/1671035460113?e=1723680000&v=beta&t=6B2ZxKbVtDEO7NOIu37XTih7MMEyOuZHsTss40QNPsM'
#         ],
#         'description': ['Sr Analytics Manager', 'Experiencia 3 años', 
#                         'Especialista en IA | Prospectiva y futuros | Generando oportunidades para mujeres en STEM | Xoogler', 
#                         'Python Backend Developer'],
#         'empresa_actual': ['No hay empresa actual', 'No hay empresa actual', 'No hay empresa actual', 'No hay empresa actual'],
#         'about': ['No hay about', 'No hay about', 'No hay about', 'No hay about'],
#         'idiomas': ['No hay idiomas', 'No hay idiomas', 'No hay idiomas', 'No hay idiomas'],
#         'conocimientos_aptitudes': ['No hay conocimientos y aptitudes', 'No hay conocimientos y aptitudes', 
#                                     'No hay conocimientos y aptitudes', 'No hay conocimientos y aptitudes'],
#         'educacion': ['No hay educacion', 'No hay educacion', 'No hay educacion', 'No hay educacion'],
#         'licencias_certificaciones': ['No hay licencias y certificaciones', 'No hay licencias y certificaciones', 
#                                       'No hay licencias y certificaciones', 'No hay licencias y certificaciones']
#     }
# df = pd.DataFrame(data)

# is_saved = save_profiles_to_db_safe(df)
# if is_saved:
#     print("Data saved successfully")
# else:
#     print("Error saving data")



    

    




Data saved successfully


In [1]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager


options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://www.linkedin.com/in/guillermo-izquierdo-colin-09586a49")

print(driver.page_source)

NameError: name 'webdriver' is not defined